<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read" data-toc-modified-id="Read-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read</a></span></li><li><span><a href="#Test-data" data-toc-modified-id="Test-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test data</a></span></li><li><span><a href="#basic" data-toc-modified-id="basic-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>basic</a></span></li><li><span><a href="#mission-dim" data-toc-modified-id="mission-dim-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>mission dim</a></span></li><li><span><a href="#Option-To-xarray-3Dim" data-toc-modified-id="Option-To-xarray-3Dim-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Option To xarray 3Dim</a></span></li><li><span><a href="#Time-dim*" data-toc-modified-id="Time-dim*-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Time dim*</a></span></li></ul></div>

In [ ]:
%pylab inline
import os
import sys
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob 

# Read

In [ ]:
path_home = "../data/extract_database_Guyane_Bresil.txt"
df = pd.read_csv(path_home, sep="|", skipinitialspace = True)
df = df.rename(columns=lambda x: x.strip())
df["num_station"] = df.num_station.str.strip()
df["mission"] = df.mission.str.strip()
df["heure"] = df.heure.str.strip()
df["jour"] = df.jour.str.strip()
df = df.replace(-999, np.nan)
df = df.loc[~df["num_station"].isnull(), ]  #type: ignore
df.loc[(df.heure.isnull())==True,"heure"] = "00:00:00"
df["time"]  = df.jour + "T" + df.heure
df["time"] = pd.to_datetime(df["time"],format='%Y-%m-%dT%H:%M:%S')
df = df[df.duplicated() == False] # 26
df = df.loc[((df.chla.isnull() == False) | (df.mes.isnull() == False) | (df.doc.isnull() == False)| (df.poc.isnull() == False)),]
df2 = df.copy()
df["coord"] = df[['lat', 'lon', 'time', 'profondeur']].apply(tuple, axis=1)
#df = df.set_index("time")

# Test data 

In [ ]:
df

In [ ]:
df.loc[(df.lat == 7.73) & (df.lon == -47.8),] #type: ignore

In [ ]:
df.loc[(df.lat == -3.628397 ) & (df.lon == -31.980950),] #type: ignore

In [ ]:
df.loc[(df.lat == -22.868450 ) & (df.lon == -43.212580 ),] #type: ignore

In [ ]:
df.loc[df.coord.duplicated()==True,] #type: ignore

# basic

In [ ]:
df2 = df2.loc[:,[ "chla", "lat", "lon", "doc", "poc", "mes", "profondeur", "time"]]
df2 = df2.set_index("time")
df2.to_xarray()

# mission dim

In [ ]:
df2.set_index(['mission', 'time'], inplace=True)
df2

In [ ]:
df2 = df2.loc[:,[ "chla", "lat", "lon", "doc", "poc", "mes", "profondeur"]]

In [ ]:
df2

In [ ]:
df3 = df2.loc[~df2.index.duplicated()]
xdf = df3.to_xarray()

In [ ]:
xdf

In [ ]:
xdf.chla[8,:]

In [ ]:
xdf.chla.sel(mission='Guyane2009')

# Option To xarray 3Dim

In [ ]:
df2["coord"] = df2[['lat', 'lon', 'time']].apply(tuple, axis=1)
df2 = df2.loc[df2.coord.duplicated()==False,]
df2 = df2.reset_index()
df2 = df2.loc[:,["lat", "lon", "time", "chla"]]

In [ ]:
df2.set_index(['lat', 'lon', 'time'], inplace=True)

In [ ]:
df2

In [ ]:
df2.to_xarray()

# Time dim*

In [ ]:
df.set_index("time", inplace=True)
df.sort_index(inplace = True)

In [ ]:

# define data with variable attributes
data_vars = {'Chla':(['time'], df.chla.values, 
                         {'units': 'mg/m3', 
                          'long_name':'Chlorophyll-a'}),
            'MES':(['time'], df.mes.values, 
                         {'units': '...', 
                          'long_name':'...'}),
            'POC':(['time'], df.poc.values, 
                         {'units': '...', 
                          'long_name':'...'}),
            'DOC':(['time'], df.doc.values, 
                         {'units': '...', 
                          'long_name':'...'}),
            'latitude':(['time'], df.lat.values, 
                         {'units': '...', 
                          'long_name':'...'}),
            'longitude':(['time'], df.lon.values, 
                         {'units': '...', 
                          'long_name':'...'}),
             'mission':(['time'], df.mission.values, 
                         {'units': '...', 
                          'long_name':'...'}),
             'num_station':(['time'], df.num_station.values, 
                         {'units': '...', 
                          'long_name':'...'}),
             'prof':(['time'], df.profondeur.values, 
                         {'units': '...', 
                          'long_name':'...'})}

# define coordinates
coords= {'time': (['time'], df.index.values)}


# define global attributes
attrs = {'creation_date':"05/06/2023", 
         'author':'Xavier Meriaux', 
         'email':'xavier.meriaux@univ-littoral.fr'}

# create dataset
ds = xr.Dataset(data_vars=data_vars, 
                coords=coords, 
                attrs=attrs)

In [ ]:
ds

In [ ]:
#ds.chla.attrs["coucou"] = "Je suis là"

In [ ]:
ds.Chla

In [ ]:
ds.Chla.sel(time=slice("2005-01-01T00:00:00", "2006-01-01")).plot(ls=":")